# huntabyte/shadcn-svelte

> 

In [ ]:
from lovely_docs.settings import settings, GitSource

from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging
import subprocess

from lovely_docs.git import clone_repo

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
build = True

source = GitSource(
    name="huntabyte/shadcn-svelte",
    doc_dir="docs/.svelte-kit/cloudflare/docs",
    repo="https://github.com/huntabyte/shadcn-svelte",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

66: 26353.0/26353.0 34.06 MiB | 875.00 KiB/ss

In [ ]:
commit, clone_dir

('489032016ddb6d312cacd5bf026505299fb3fcd4',
 PosixPath('../git_dir/github.com/huntabyte/shadcn-svelte'))

In [ ]:
if build:
    with open("huntabyte-shadcn-svelte.log", "w") as log_file:
        # Clean the repo
        subprocess.run(["git", "clean", "-dfx"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in root
        subprocess.run(["pnpm", "install"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Build the package and docs
        subprocess.run(["pnpm", "build"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # # Install dependencies in docs
        # subprocess.run(["pnpm", "install"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

        # # Build the docs
        # subprocess.run(["pnpm", "build"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

In [ ]:
# Path("shadcn-svelte.json").write_text(tree.model_dump_json(indent=2))

In [ ]:
# tree.subdirs = tree.subdirs[:3]
# tree.subdirs[1].subdirs = tree.subdirs[1].subdirs[:4]

#### Split the single llms.txt file into pages and directories based on heading levels

In [ ]:
# compacted_tree = compact_tree(tree)

In [ ]:
# Path("shadcn-svelte-compact.json").write_text(compacted_tree.model_dump_json(indent=2))

In [ ]:
extra = """In cli examples with multipls pms, only keep one for npm.
For component install, add -y -o to the example with explanation:
  -y: skip confirmation prompt (default: false)
  -o: overwrite existing files (default: false)
`npx shadcn-svelte@latest add <component> -y -o`
"""

processed_tree = await process_tree_depth_first(settings, tree, source.name, extra, extra)

+216.527s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/accordion.md
+0.317s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/alert-dialog.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/alert.md
+0.016s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/aspect-ratio.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/avatar.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/badge.md
+0.016s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/breadcrumb.md
+0.017s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/button-group.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing components/button.md
+0.015s DEBUG preprocessor/nbs/lovely_d

In [ ]:
# Path("shadcn-svelte-processed.json").write_text(processed_tree.model_dump_json(indent=2))

#### Calculate cost

In [ ]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 401,052
  Output tokens: 148,786
  Total tokens: 549,838
Cost:
  Total: $1.14
  Input: $0.40
  Output: $0.74


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)

In [ ]:
processed_tree.children

[DocItem(origPath=PosixPath('components'), name='components', displayName='Components', digest='# Components\n\nComprehensive collection of 60+ reusable UI components built on Bits UI, Embla Carousel, LayerChart, and other libraries. Each component is installable via CLI and composable with subcomponents.\n\n## Installation Pattern\n\nAll components install via:\n```bash\nnpx shadcn-svelte@latest add <component> -y -o\n```\nFlags: `-y` skips confirmation, `-o` overwrites existing files.\n\n## Core Components\n\n**Accordion** - Vertically stacked interactive headings with single/multiple open items. Root/Item/Trigger/Content subcomponents. WAI-ARIA accessible.\n\n**Alert** - Callout component with Root/Title/Description. Supports default and destructive variants.\n\n**Alert Dialog** - Modal dialog interrupting user with important content. Root/Trigger/Content/Header/Title/Description/Footer/Cancel/Action subcomponents.\n\n**Avatar** - Image element with fallback text. Root/Image/Fallbac